In [ ]:
!pip install torchinfo

In [13]:
import torch
import torchinfo
import cv2
from ultralytics import YOLO


In [20]:
# load custom yolov5 model
# model = torch.hub.load('ultralytics/yolov8', "custom", path='../ml_models/model_100epochs_v1.pt', force_reload=True).autoshape()
checkpoint = YOLO('../ml_models/model_100epochs_v1.pt')
model = checkpoint.model
# show model information
torchinfo.summary(checkpoint.model, input_size=(1, 3, 640, 640), col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"), depth=4)
# print(checkpoint)

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
DetectionModel                                     [1, 3, 640, 640]          [1, 5, 8400]              --                        --                        --
├─Sequential: 1-1                                  --                        --                        --                        --                        --
│    └─Conv: 2-1                                   [1, 3, 640, 640]          [1, 32, 320, 320]         --                        --                        --
│    │    └─Conv2d: 3-1                            [1, 3, 640, 640]          [1, 32, 320, 320]         (864)                     [3, 3]                    88,473,600
│    │    └─BatchNorm2d: 3-2                       [1, 32, 320, 320]         [1, 32, 320, 320]         (64)                      --                        64
│    └─Detect: 2-96                  

In [21]:
# draw boxes around detected objects
def analyze_image(model, frame):
    results = model(frame)

    label, coord = results.xyxy[0][:, -1], results.xyxy[0][:, :-1]

    x_shape, y_shape = frame.shape[1], frame.shape[0]
    print("len label", len(label))
    if (len(label) > 0):
        row = coord[0]
        x1, y1, x2, y2 = int(row[0] * x_shape), int(row[1] * y_shape), int(row[2] * x_shape), int(row[3] * y_shape)
        bgr = (0, 255, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
        print(label, coord)
    # for i in range(len(label)):
    #     row = coord[i]
    #     x1, y1, x2, y2 = int(row[0] * x_shape), int(row[1] * y_shape), int(row[2] * x_shape), int(row[3] * y_shape)
    #     bgr = (0, 255, 0)
    #     cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
    #     print(label,coord)

    cv2.imshow('frame', frame)


In [22]:
cap = cv2.VideoCapture('../data/video.mp4')
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 640))
    analyze_image(model, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
